In [1]:
!pip install -r requirements.txt
!apt install -y libsm6 libfontconfig1 libxrender1 libxext6

     |████████████████████████████████| 634kB 1.6MB/s eta 0:00:01
     |████████████████████████████████| 28.3MB 1.3MB/s eta 0:00:01    |█████████▊                      | 8.6MB 7.9MB/s eta 0:00:03     |█████████████████████████▏      | 22.3MB 4.5MB/s eta 0:00:02
     |████████████████████████████████| 2.1MB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.1MB/s eta 0:00:011
     |████████████████████████████████| 26.1MB 12.0MB/s eta 0:00:01    |███▊                            | 3.0MB 12.7MB/s eta 0:00:02
     |████████████████████████████████| 26.5MB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 5.9MB/s eta 0:00:01
     |████████████████████████████████| 7.0MB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 235kB 5.4MB/s eta 0:00:01
     |███

     |████████████████████████████████| 40kB 3.0MB/s  eta 0:00:01
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654976 sha256=a3e2134dc3b36acdaaa14368a48d0aa12924e94d793fb5b2aef5f5dc94dc303b
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=3891d75e871da584bd4816574443286991f01cbba535e005ffba169835d3043a
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built imgaug sklearn
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
np.random.seed(111)
import os
import json
from yolo.frontend import create_yolo, get_object_labels

In [2]:
def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "config.json"))
    
    return config, os.path.join(dirname, "weights.h5")

In [3]:
import warnings

config_file = "configs/brio.json"
config, weight_file = setup_training(config_file)

if config['train']['is_only_detect']:
    labels = ["object"]
else:
    if config['model']['labels']:
        labels = config['model']['labels']
    else:
        labels = get_object_labelsget_object_labels(config['train']['train_annot_folder'])
        
print(labels)

# 1. Construct the model 
yolo = create_yolo(config['model']['architecture'],
                   labels,
                   config['model']['input_size'],
                   config['model']['anchors'],
                   config['model']['coord_scale'],
                   config['model']['class_scale'],
                   config['model']['object_scale'],
                   config['model']['no_object_scale'])

# 2. Load the pretrained weights (if any) 
yolo.load_weights(config['pretrained']['full'], by_name=True)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # 3. actual training 
    yolo.train(config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['actual_epoch'],
               weight_file,
               config["train"]["batch_size"],
               config["train"]["jitter"],
               config['train']['learning_rate'], 
               config['train']['train_times'],
               config['train']['valid_times'],
               config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['first_trainable_layer'],
               config['train']['is_only_detect'])


brio is already exists. Weight file in directory will be overwritten
Weight file and Config file will be saved in "brio"
['brio']
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenet_0.75_224 (Model)   (None, 7, 7, 768)         1832976   
_________________________________________________________________
detection_layer_30 (Conv2D)  (None, 7, 7, 30)          23070     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 5, 6)        0         
Total params: 1,856,046
Trainable params: 1,839,630
Non-trainable params: 16,416
_________________________________________________________________
Fail to load pre-trained weights. Make sure weight file path.
Epoch 1/20
82/82 [==============================] 